In [1]:
import pandas as pd

# READ DATA

champion index 

In [2]:
champion = pd.read_csv('data/clean/champion.csv',sep=',')

In [3]:
champion.head()

,championId,championName
0,23,Tryndamere
1,203,Kindred
2,90,Malzahar
3,145,Kaisa
4,12,Alistar


In [4]:
match = pd.read_csv('data/clean/match.csv',sep=',')

In [5]:
match.head()

,id,gameVersion,gameDurationCluster,afkEarlySurrender,tierId,divisionId
0,0,13.19,1,False,4,4
1,1,13.19,2,False,4,4
2,2,13.13,1,False,4,4
3,3,13.13,2,False,4,4
4,4,13.19,1,False,4,4


In [6]:
participant = pd.read_csv('data/clean/participant.csv',sep=',')

In [7]:
participant.head()

,id,matchId,teamId,turn,gameEndedInEarlySurrender,gameEndedInSurrender,lane,role,championId,champExperience,...,neutralMinionsKilled,totalMinionsKilled,visionScore,visionWardsBoughtInGame,detectorWardsPlaced,wardsKilled,wardsPlaced,objectivePlaystyle,healPlaystyle,Position
0,0,0,0,1,False,False,JUNGLE,NONE,23,13182,...,4,203,10,0,0,0,7,0,4,TOP
1,1,0,0,2,False,False,JUNGLE,NONE,203,12374,...,145,30,13,0,0,4,1,0,3,JUNGLE
2,2,0,0,3,False,False,MIDDLE,SOLO,90,13929,...,4,221,16,0,0,1,8,0,1,MIDDLE
3,3,0,0,4,False,False,BOTTOM,CARRY,145,9258,...,8,151,15,0,0,3,7,2,1,BOTTOM
4,4,0,0,5,False,False,BOTTOM,SUPPORT,12,8200,...,0,33,45,9,6,3,22,2,4,UTILITY


In [8]:
team = pd.read_csv('data/clean/team.csv',sep=',')

In [9]:
team.head()

,id,matchId,sideId,baronFirst,baronKills,championFirst,championKills,dragonFirst,dragonKills,inhibitorFirst,inhibitorKills,riftHeraldFirst,riftHeraldKills,towerFirst,towerKills,win
0,0,0,100,True,1,False,18,False,0,False,1,True,1,False,6,False
1,1,0,200,False,0,True,35,True,4,True,2,False,1,True,9,True
2,2,1,100,True,1,False,48,True,2,True,2,False,1,False,9,True
3,3,1,200,False,1,True,45,False,2,False,0,True,1,True,5,False
4,4,2,100,False,0,True,26,False,0,False,0,True,1,True,3,False


# model of prediction of victory

## transform data for train

In [10]:
team['sideId'].value_counts()
team['sideId']=team['sideId'].replace({
    100:0,
    200:1
})

cambiaremos sideId por un valor mas diferenciador ya que significa que equipo pertence cada integrante

In [32]:
participant_train  = participant[['id','teamId','championBan','championId','champExperience','Position']].head(10000)

In [33]:
participant_train.columns.tolist()

['id', 'teamId', 'championBan', 'championId', 'champLevel', 'Position']

In [34]:
participant_train

,id,teamId,championBan,championId,champLevel,Position
0,0,0,-1,23,15,TOP
1,1,0,30,203,14,JUNGLE
2,2,0,107,90,15,MIDDLE
3,3,0,24,145,12,BOTTOM
4,4,0,119,12,11,UTILITY
...,...,...,...,...,...,...
9995,9995,1999,36,1,12,TOP
9996,9996,1999,119,120,14,JUNGLE
9997,9997,1999,59,34,11,MIDDLE
9998,9998,1999,895,429,9,BOTTOM


In [35]:

team_num = participant_train['teamId'].unique()
len(team_num)

    

2000

In [36]:
teams_dfs = pd.DataFrame()
#participant_train.set_index('id', inplace=True)
for i in range(len(team_num)):
    team_df = participant_train[participant_train['teamId'] == i][['championId','champExperience','championBan','Position']]
    df = pd.DataFrame()
    for j in range(len(team_df)):
        df = pd.concat([df,team_df.iloc[j]])
    df = df.T
    df['teamId'] = i
    teams_dfs = pd.concat([teams_dfs,df])
    
teams_dfs.set_index('teamId', inplace=True)

teams_dfs



,championId,champLevel,championBan,Position,championId,champLevel,championBan,Position,championId,champLevel,championBan,Position,championId,champLevel,championBan,Position,championId,champLevel,championBan,Position
teamId,,,,,,,,,,,,,,,,,,,,
0,23,15,-1,TOP,203,14,30,JUNGLE,90,15,107,MIDDLE,145,12,24,BOTTOM,12,11,119,UTILITY
1,122,12,200,TOP,104,15,91,JUNGLE,34,14,30,MIDDLE,29,14,157,BOTTOM,412,12,233,UTILITY
2,777,17,360,TOP,56,16,221,JUNGLE,268,17,238,MIDDLE,29,16,107,BOTTOM,555,14,54,UTILITY
3,166,18,53,TOP,233,16,8,JUNGLE,134,17,24,MIDDLE,145,17,200,BOTTOM,143,14,107,UTILITY
4,98,10,555,TOP,28,11,107,JUNGLE,238,13,-1,MIDDLE,21,11,203,BOTTOM,26,9,421,UTILITY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,34,15,53,MIDDLE,11,14,51,JUNGLE,92,15,64,TOP,222,12,238,BOTTOM,888,12,360,UTILITY
1996,887,13,119,TOP,233,12,-1,JUNGLE,84,13,238,MIDDLE,81,12,24,BOTTOM,235,10,101,UTILITY
1997,86,15,360,TOP,33,13,555,JUNGLE,517,15,876,MIDDLE,115,13,164,BOTTOM,43,11,238,UTILITY


In [37]:
columns = ['championId_1','champLevel_1','championBan_1','Position_1','championId_2','champLevel_2','championBan_2','Position_2','championId_3','champLevel_3','championBan_3','Position_3','championId_4','champLevel_4','championBan_4','Position_4','championId_5','champLevel_5','championBan_5','Position_5']

In [38]:
teams_dfs.columns = columns

In [39]:
teams_dfs

,championId_1,champLevel_1,championBan_1,Position_1,championId_2,champLevel_2,championBan_2,Position_2,championId_3,champLevel_3,championBan_3,Position_3,championId_4,champLevel_4,championBan_4,Position_4,championId_5,champLevel_5,championBan_5,Position_5
teamId,,,,,,,,,,,,,,,,,,,,
0,23,15,-1,TOP,203,14,30,JUNGLE,90,15,107,MIDDLE,145,12,24,BOTTOM,12,11,119,UTILITY
1,122,12,200,TOP,104,15,91,JUNGLE,34,14,30,MIDDLE,29,14,157,BOTTOM,412,12,233,UTILITY
2,777,17,360,TOP,56,16,221,JUNGLE,268,17,238,MIDDLE,29,16,107,BOTTOM,555,14,54,UTILITY
3,166,18,53,TOP,233,16,8,JUNGLE,134,17,24,MIDDLE,145,17,200,BOTTOM,143,14,107,UTILITY
4,98,10,555,TOP,28,11,107,JUNGLE,238,13,-1,MIDDLE,21,11,203,BOTTOM,26,9,421,UTILITY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,34,15,53,MIDDLE,11,14,51,JUNGLE,92,15,64,TOP,222,12,238,BOTTOM,888,12,360,UTILITY
1996,887,13,119,TOP,233,12,-1,JUNGLE,84,13,238,MIDDLE,81,12,24,BOTTOM,235,10,101,UTILITY
1997,86,15,360,TOP,33,13,555,JUNGLE,517,15,876,MIDDLE,115,13,164,BOTTOM,43,11,238,UTILITY


In [44]:
teams_dfs_merge_team=teams_dfs.merge(team, left_index=True, right_index=True)

In [45]:
teams_dfs_merge_team

,championId_1,champLevel_1,championBan_1,Position_1,championId_2,champLevel_2,championBan_2,Position_2,championId_3,champLevel_3,...,championKills,dragonFirst,dragonKills,inhibitorFirst,inhibitorKills,riftHeraldFirst,riftHeraldKills,towerFirst,towerKills,win
0,23,15,-1,TOP,203,14,30,JUNGLE,90,15,...,18,False,0,False,1,True,1,False,6,False
1,122,12,200,TOP,104,15,91,JUNGLE,34,14,...,35,True,4,True,2,False,1,True,9,True
2,777,17,360,TOP,56,16,221,JUNGLE,268,17,...,48,True,2,True,2,False,1,False,9,True
3,166,18,53,TOP,233,16,8,JUNGLE,134,17,...,45,False,2,False,0,True,1,True,5,False
4,98,10,555,TOP,28,11,107,JUNGLE,238,13,...,26,False,0,False,0,True,1,True,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,34,15,53,MIDDLE,11,14,51,JUNGLE,92,15,...,29,False,2,True,1,True,2,True,9,True
1996,887,13,119,TOP,233,12,-1,JUNGLE,84,13,...,10,False,1,False,0,False,0,False,0,False
1997,86,15,360,TOP,33,13,555,JUNGLE,517,15,...,28,True,2,True,1,True,2,True,9,True
1998,164,14,107,TOP,76,12,36,JUNGLE,103,12,...,21,False,2,True,2,True,2,True,9,True
